In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%ls

drive/  sample_data/


In [4]:
%cd /content/drive/MyDrive/PatchTST

/content/drive/MyDrive/PatchTST


In [5]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

### argument 수작업으로 객체 생성

In [6]:
arg_seq_len=336
arg_pred_len = 96
model_name='PatchTST'

root_path_name='./dataset/'
data_path_name='ETTh1.csv'
model_id_name='ETTh1'
data_name='ETTh1'

random_seed=2021

class Args:
    random_seed = 2021

    is_training = 1
    model_id = model_id_name
    model = model_name

    data = data_name
    root_path = root_path_name
    data_path = data_path_name
    features = 'M'
    target = 'OT'
    freq = 'h'
    checkpoints = './checkpoints/'

    seq_len = arg_seq_len
    label_len = 48
    pred_len = arg_pred_len

    fc_dropout= 0.05
    head_dropout = 0.0
    patch_len = 16
    stride = 8
    padding_patch = 'end'
    revin = 1
    affine = 0
    subtract_last = 0
    decomposition = 0
    kernel_size = 25
    individual = 0

    embed_type = 0
    enc_in = 7
    dec_in = 7
    c_out = 7
    d_model = 512
    n_heads = 8
    e_layers = 2
    d_layers = 1
    d_ff = 2048
    moving_avg = 25
    factor = 1
    distil = True
    dropout = 0.05
    embed = 'timeF'
    activation = 'gelu'
    output_attention = False
    do_predict = True

    num_workers = 10
    itr = 2
    train_epochs = 100
    batch_size = 128
    patience = 100
    learning_rate = 0.0001
    des = 'test'
    loss = 'MSE'
    lradj = 'type1'
    pct_start = 0.3
    use_amp = False

    use_gpu = True
    gpu = 0
    use_multi_gpu = False
    devices = '0'
    test_flop = False

args = Args();

# random seed
fix_seed = args.random_seed
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Args in experiment:


### 실험 시작

In [ ]:
Exp = Exp_Main

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des,ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        if args.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.predict(setting, True)

        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model_id,
                                                                                                args.model,
                                                                                                args.data,
                                                                                                args.features,
                                                                                                args.seq_len,
                                                                                                args.label_len,
                                                                                                args.pred_len,
                                                                                                args.d_model,
                                                                                                args.n_heads,
                                                                                                args.e_layers,
                                                                                                args.d_layers,
                                                                                                args.d_ff,
                                                                                                args.factor,
                                                                                                args.embed,
                                                                                                args.distil,
                                                                                                args.des, ii)

    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()


Use GPU: cuda:0
>>>>>>>start training : ETTh1_PatchTST_ETTh1_ftM_sl336_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8209
val 2785
test 2785
Epoch: 1 cost time: 31.485963582992554
Epoch: 1, Steps: 64 | Train Loss: 0.5166196 Vali Loss: 0.8070070 Test Loss: 0.4650646
Validation loss decreased (inf --> 0.807007).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 cost time: 28.69879150390625
